# VFF Case Study

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [ ]:
# Read the data
exhibit_06 = pd.read_csv('Exhibit 6.csv')
exhibit_06['Date'] = pd.to_datetime(exhibit_06['Date'])
exhibit_06.head()

,Date,Minimum Price,Maximum Price,Modal Price
0,2015-04-01,1000,1500,1300
1,2015-04-02,1000,1500,1300
2,2015-04-03,1200,1800,1600
3,2015-04-04,1000,2500,1300
4,2015-04-06,1000,1800,1500


In [ ]:
# Read the data
exhibit_07 = pd.read_excel('Exhibit 7.xlsx')
exhibit_07['Date'] = pd.to_datetime(exhibit_07['Date'])
exhibit_07['Month'] = exhibit_07['Date'].dt.month

exhibit_07.head() 

,Date,Purchase Units,Wastage Units,Retail Sales Units,Wholesale Sales,Total Sales Units,Closing Inventory,Month
0,2015-04-01,42,6,26,10,36,6,4
1,2015-04-02,47,20,23,4,27,20,4
2,2015-04-03,53,0,31,26,57,0,4
3,2015-04-04,101,45,41,15,56,45,4
4,2015-04-05,73,22,44,7,51,22,4


## Question 01
What are the typical post harvest concerns for farmers and what is the role of a collection center in integrating farm supply chains and improving fresh produce distribution ? How does the supply chain network for VFF compare with traditional fresh produce distribution?

**Post-harvest concerns for farmers :**
Post-harvest concerns for farmers include the preservation of crop quality, maintaining freshness, preventing spoilage, minimizing losses, and ensuring timely transportation and delivery to markets. 

**Role of a collection center :**
A collection center plays a crucial role in integrating farm supply chains and improving fresh produce distribution. It serves as an aggregation point for farmers to bring their produce, where it is sorted, graded, and packaged for transportation to markets. The collection center also provides farmers with access to market information, facilitates negotiations with buyers, and offers training on best post-harvest handling practices. By streamlining the collection and distribution process, collection centers help to reduce post-harvest losses, improve the quality of fresh produce, and increase farmers' income.

**Supply chain network for VFF v.s Traditional fresh produce distribution**
The supply chain network for VFF differs from traditional fresh produce distribution in that it is a direct-to-consumer model. VFF connects farmers directly with consumers through the collection center, eliminating intermediaries and reducing the time and costs associated with traditional supply chain networks. By bypassing traditional supply chain intermediaries, VFF can offer farmers better prices for their produce while providing consumers with fresher, locally sourced products. Additionally, VFF's supply chain network can help to reduce food waste by providing consumers with more accurate information about the availability of fresh produce and reducing the need for excess inventory.

## Question 02
What profit does the farmer earn on produce and how does this compare with the actual customer purchase price from a retail outlet ?

In [ ]:
# Farmer Profit 
print(f'The farmer earn {10-1.25-0.25-0.25} per kilogram on produce.')

The farmer earn 8.25 per kilogram on produce.


Assuming the selling price follows the uniform distribution between 20 and 25, we can tell that the selling price is twice more than the profit earn by farmers on produce.

## Question 03
List the produce attributes and other variables influencing demand for produce that affect the quantity to be procured by VFF.

The demand for produce and the quantity to be procured by VFF can be influenced by various attributes and variables such as:

1. Quality: The quality of the produce is a crucial factor that can significantly impact demand. Produce that is fresh, visually appealing, and free from defects is more likely to be in demand.
2. Temperature control: As F&V are sensitive to heat, proper temperature control during transportation and storage is necessary to prevent spoilage and increase the shelf life of the produce.
3. Handling: Over-handling or inappropriate handling of produce can lead to spoilage, bruising, and other damage that can decrease the quality of the produce and reduce demand.
4. Moisture loss: Moisture loss is a common cause of wastage in produce. VFF needs to ensure that the produce is stored and transported under appropriate conditions to minimize moisture loss.
5. Packaging: Poor packaging can also lead to spoilage and wastage of produce. Adequate packaging can help to protect the produce from damage and extend its shelf life.
6. Storage conditions: Different types of produce require different storage techniques and temperatures. Proper storage conditions are necessary to prevent spoilage and maintain the quality of the produce.
7. Overestimation of demand: Overestimation of demand can result in excess inventory, which can lead to spoilage and wastage of produce. VFF needs to accurately estimate demand to prevent excess inventory and minimize wastage.
8. Type of produce: Different types of produce have different characteristics and requirements. VFF needs to consider the specific attributes of each type of produce to determine the quantity to be procured and ensure that it is of high quality and in demand.

## Question 04
How should Vasant Farm Fresh (VFF) reduce food wastage ? Consider three different scenarios:

We first calculate the current wastage ratio for the later comparisons.

In [ ]:
# Original wastage ratio
(exhibit_07.groupby('Month')['Purchase Units'].sum() - exhibit_07.groupby('Month')['Total Sales Units'].sum()) / exhibit_07.groupby('Month')['Purchase Units'].sum()

Month
4    0.443697
5    0.583266
dtype: float64

### (a)
Determine the optimal procurement quantity for cauliflower assuming that the average cost price and average selling price take fixed values and shelf life in one day.

In [ ]:
# Average Sales
avg_purchase = exhibit_07['Purchase Units'].mean()
# Standard Deviation
std_purchase = exhibit_07['Purchase Units'].std()
# Critial Ratio 
co_01 = 9.14
cu_01 = 8.65
cr_01 = cu_01/(co_01+cu_01)
# Z-score
z_01 = stats.norm.ppf(cr_01, loc = 0, scale = 1)
# Optimal Quantity
Q_star_01 = round(z_01 * std_purchase + avg_purchase)
print(f"The optimal procurement quantity for cauliflower is {Q_star_01}.")

The optimal procurement quantity for cauliflower is 49.


In [ ]:
# Calculate the daily wastage
exhibit_07['wastageA'] = exhibit_07.apply(
    lambda row: 0 if Q_star_01 - row['Total Sales Units'] < 0 else Q_star_01 - row['Total Sales Units'], axis=1)

In [ ]:
# Calculate wastage ratio in the first case
exhibit_07.groupby('Month')['wastageA'].sum() / (exhibit_07.groupby('Month')['Date'].count()*Q_star_01)

Month
4    0.336054
5    0.671494
dtype: float64

### (b)
Determine the optimal procurement quantity by using daily price point for purchases and sales. Assume a one day shelf life.

In [ ]:
# Assume a constant salvage price of Rs 5 /kg of cauliflower. 
# Assume a selling price of U[20, 25] (i.e. The selling price comes from a uniform distribution  between 20 and 25 )
# To get the purchase price, fit a normal distribution to the modal price you see in the table of Exhibit 6.

# Generate a sample for selling price of U[20,25]
import random
from scipy.stats import norm

random.seed(716)
sample_size = 52
exhibit_06['Selling Price'] = [random.uniform(20, 25) for _ in range(sample_size)]

# Fit the normal distribution to the modal price
mu, std = norm.fit(exhibit_06['Modal Price']/100)
exhibit_06['Modal Price Fitted'] = np.random.normal(mu, std, size = sample_size)

# Calculate
exhibit_06['cu_02'] = exhibit_06['Selling Price'] - exhibit_06['Modal Price Fitted']
exhibit_06['co_02'] = exhibit_06['Modal Price Fitted'] - 5
exhibit_06['cr_02'] = exhibit_06['cu_02']/(exhibit_06['co_02']+exhibit_06['cu_02'])

# Z-score
exhibit_06['z_02'] = stats.norm.ppf(exhibit_06['cr_02'], loc = 0, scale = 1)

# Optimal Quantity
exhibit_06['Q_star_b'] = round(exhibit_06['z_02'] * std_purchase + avg_purchase,0)

In [ ]:
# Wastage to be calculated when we write report
a = exhibit_06[['Date','Q_star_b']]
b = exhibit_07[['Date','Total Sales Units','Purchase Units']]
merged_df = pd.merge(a, b, on = 'Date', how = 'outer')

# Fill the empty data rows
merged_df['Q_star_b'] = merged_df.sort_values('Date')['Q_star_b'].ffill(axis = 0)
merged_df['wastageB'] = merged_df.apply(
    lambda row: 0 if row['Q_star_b'] - row['Total Sales Units'] < 0 else row['Q_star_b'] - row['Total Sales Units'], axis=1)

In [ ]:
# Optimized wastage ratio assuming a one day shelf life
merged_df['Month'] = merged_df['Date'].dt.month
merged_df.groupby('Month')['wastageB'].sum() / merged_df.groupby('Month')['Q_star_b'].sum()

Month
4    0.361072
5    0.667099
dtype: float64

### (c)
Determine the optimal procurement quantity by using daily price point for purchases and sales. Assume a two day shelf life.

In [ ]:
merged_df['Leftover'] = merged_df['Q_star_b'] - merged_df['Total Sales Units']
merged_df['Lag_Leftover'] = merged_df['Leftover'].shift(1).fillna(0)

# Assume all produce follow FIFO method (ex. The leftover in the first day should be sold at very first in the second day.)
def cal_wastageC(row):
  if row['Lag_Leftover'] > row['Total Sales Units']:
    return row['Lag_Leftover'] - row['Total Sales Units']
  else:
    return 0

merged_df['wastageC'] = merged_df.apply(cal_wastageC, axis=1)

In [ ]:
# Optimized wastage ratio assuming a two day shelf life
merged_df.groupby('Month')['wastageC'].sum() / merged_df.groupby('Month')['Q_star_b'].sum()

Month
4    0.066291
5    0.356262
dtype: float64

### Summary of the wastage ratio:

|      |Original|Case A|Case B|Case C|
|------|------|------|------|------|
|**April**|0.443697|0.336054|0.361072|0.066291|
|**May**|0.583266|0.671494|0.667099|0.356262|

## Question 05

What other decision areas should VVF be concerned about ?

- Supply chain management: optimizing transportation flow to reduce waste and increase profit
- Guiding the farmers: analyzing customer preferences to help farmers grow the most popular fruits and vegetables, which reduces waste and meets customer demand
- Quality enhancement: setting standards and procedures for handling, packaging, and transporting produce to ensure it is delivered in pristine condition, which can increase customer loyalty and satisfaction
- New product development and innovation: exploring new types of fruits and vegetables that are not yet in the market, experimenting with new farming techniques to improve efficiency and yield, and staying ahead of the competition by diversifying the product line and keeping up with the latest trends and technology.